In [1]:
import os
os.environ['FLYTE_PLATFORM_URL'] = "flyte-staging.lyft.net"
from flytekit.sdk.types import Types
from flytekit.common import utils
from flytekit.common.workflow_execution import SdkWorkflowExecution

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
wf = SdkWorkflowExecution.fetch(project="flytekubecondemo2019", domain="development", name="gt1785aotr")
print(wf.outputs)

{'predictions': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [3]:
wf = SdkWorkflowExecution.fetch(project="flytekubecondemo2019", domain="development", name="f1487e14d159749d0a9f")
print(wf.outputs["confusion_matrix_image"])

metadata {
  type {
  }
}
uri: "s3://lyft-modelbuilder/fg/50822df4ba23a48f29afdf81aa32d326"



In [4]:
local_file = "/tmp/cm-4.png"
wf.outputs["confusion_matrix_image"].download(local_file)
from IPython.display import Image
Image(filename=local_file)

FlyteAssertion: Cannot download blob to a location that already exists when overwrite is not set to True.  Attempted download from s3://lyft-modelbuilder/fg/50822df4ba23a48f29afdf81aa32d326 -> /tmp/cm-4.png

In [9]:
from workflows.classifier_train_workflow import train_lp
from flytekit.common.launch_plan import SdkLaunchPlan

PROJECT="flytekubecondemo2019"
dataprep_wf_exec = SdkWorkflowExecution.fetch(project=PROJECT, domain="development", name="u86fgpn177-prepare-0")
frames_mpblobs = dataprep_wf_exec.outputs["selected_frames_mpblobs"]
streams_names = dataprep_wf_exec.outputs["streams_names_out"]

DEFAULT_TRAINING_VALIDATION_CONFIG_FILE = "models/classifier/resnet50/configs/model_training_config_demo.json"

train_lp = SdkLaunchPlan.fetch(
    project=PROJECT, 
    domain="development", 
    name="workflows.classifier_train_workflow.ClassifierTrainWorkflow",
    version="e0269ce34f2a493bd54953c56b3fb6fd08aa8808")

train_lp.execute(
    project=PROJECT,
    domain="development",
    inputs={
        "available_streams_mpblobs": frames_mpblobs,
        "available_streams_names": streams_names,
        "streams_metadata_path": "s3://lyft-modelbuilder/metadata/_FlyteKubeconDemo2019Dataset/metadata/streams_metadata.json",
    }
)

KeyboardInterrupt: 